In [6]:
import matplotlib.pyplot as plt
import torch
import torch_geometric as pyg
import torch.nn as nn
from torch_geometric.data import Data
from numpy.linalg import eigh
from scipy.sparse.linalg import eigsh
from torch_geometric.utils import to_scipy_sparse_matrix
import copy

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import to_scipy_sparse_matrix
import numpy as np
import scipy.sparse as sp


# find device
if torch.cuda.is_available(): # NVIDIA
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # apple M1/M2
    device = torch.device('mps')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [7]:
dataset = pyg.datasets.LRGBDataset(root='dataset/peptides-func', name="Peptides-func")

# Check if dataset has splits; if not, create them manually
if hasattr(dataset, 'train_val_test_idx'):
    peptides_train = dataset[dataset.train_val_test_idx['train']]
    peptides_val = dataset[dataset.train_val_test_idx['val']]
    peptides_test = dataset[dataset.train_val_test_idx['test']]
else:
    # Create train, val, test splits manually
    num_train = int(0.8 * len(dataset))
    num_val = int(0.1 * len(dataset))
    num_test = len(dataset) - num_train - num_val
    peptides_train, peptides_val, peptides_test = torch.utils.data.random_split(dataset, [num_train, num_val, num_test])

batch_size = 32
train_loader = pyg.loader.DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = pyg.loader.DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = pyg.loader.DataLoader(peptides_test, batch_size=batch_size, shuffle=False)

# Check number of classes and label distribution
if hasattr(dataset, 'num_tasks'):
    num_classes = dataset.num_tasks
elif hasattr(dataset, 'num_classes'):
    num_classes = dataset.num_classes
else:
    # Assume binary classification if not specified
    num_classes = 1
print(f"Number of classes: {num_classes}")

all_labels = np.concatenate([data.y.numpy() for data in dataset], axis=0)
label_distribution = np.mean(all_labels, axis=0)
print(f"Label distribution: {label_distribution}")

Number of classes: 10
Label distribution: [0.08884393 0.03540881 0.06419571 0.06226432 0.6272418  0.19755358
 0.10687023 0.18412581 0.01995769 0.2598179 ]


In [8]:
def compute_laplace_pe(data, num_eigenvec=10):
    G = to_networkx(data, to_undirected=True)
    A = nx.adjacency_matrix(G).astype(float)
    num_nodes = A.shape[0]
    D = np.diag(np.array(A.sum(axis=1)).flatten())
    L = D - A.todense()
    L = torch.tensor(L, dtype=torch.float, device=device)
    try:
        eigenvalues, eigenvectors = torch.linalg.eigh(L)
    except RuntimeError:
        eigenvalues, eigenvectors = torch.symeig(L, eigenvectors=True)
    available_eigenvec = eigenvectors.shape[1] - 1
    actual_num_eigenvec = min(num_eigenvec, available_eigenvec)
    eigenvectors = eigenvectors[:, 1:1 + actual_num_eigenvec]
    if actual_num_eigenvec < num_eigenvec:
        pad_size = num_eigenvec - actual_num_eigenvec
        padding = torch.zeros(eigenvectors.shape[0], pad_size, device=device)
        eigenvectors = torch.cat([eigenvectors, padding], dim=1)
    return eigenvectors  # Shape: (num_nodes, num_eigenvec)

# Random Walk Structural Embeddings (RWSE)
def compute_rwse(data, walk_length=10):
    G = to_networkx(data, to_undirected=True)
    A = nx.adjacency_matrix(G).astype(float)
    A = A.todense()
    num_nodes = A.shape[0]
    A = torch.tensor(A, dtype=torch.float, device=device)
    rw_features = []
    A_power = A.clone()
    for _ in range(walk_length):
        diag = torch.diagonal(A_power)
        rw_features.append(diag)
        A_power = torch.matmul(A_power, A)
    rwse = torch.stack(rw_features, dim=1)  # (num_nodes, walk_length)
    return rwse  # Shape: (num_nodes, walk_length)

# SignNet to ensure sign invariance
class SignNet(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(SignNet, self).__init__()
        self.phi = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.ReLU(),
            nn.Linear(out_dim, out_dim)
        )

    def forward(self, x):
        return self.phi(x) + self.phi(-x)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import TransformerConv, global_mean_pool
from torch.optim import Adam
import numpy as np

# ----------------------------
# 1. Define the Graph Transformer Model
# ----------------------------
class GraphTransformerModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout=0.5):
        """
        A simple 2-layer Transformer-based GNN.

        :param in_channels:  Number of input features per node
        :param hidden_channels: Hidden layer size
        :param out_channels:   Number of prediction classes (or tasks)
        :param heads:          Number of attention heads in TransformerConv
        :param dropout:        Dropout probability
        """
        super(GraphTransformerModel, self).__init__()
        # First transformer layer
        self.conv1 = TransformerConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        # After conv1, the output dimension is hidden_channels * heads
        self.conv2 = TransformerConv(hidden_channels * heads, hidden_channels, heads=1, dropout=dropout)
        
        # Final linear layer for graph-level prediction
        self.lin = nn.Linear(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index, batch):
        """
        Forward pass of the model.
        :param x:         Node features [num_nodes, in_channels]
        :param edge_index: Edge indices [2, num_edges]
        :param batch:     Batch indices for each node [num_nodes]
        :return:          Model output (logits), shape [batch_size, out_channels]
        """
        # First transformer block
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        # Second transformer block
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        # Global average pooling over the nodes in each graph
        x = global_mean_pool(x, batch)
        
        # Optionally apply dropout before the final linear layer
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Final classification/regression head
        x = self.lin(x)
        return x

# ----------------------------
# 2. Prepare your dataset & loaders (from your snippet)
# ----------------------------
import torch_geometric as pyg

import torch
import numpy as np

dataset = pyg.datasets.LRGBDataset(root='dataset/peptides-func', name="Peptides-func")

# Check if dataset has splits; if not, create them manually
if hasattr(dataset, 'train_val_test_idx'):
    peptides_train = dataset[dataset.train_val_test_idx['train']]
    peptides_val = dataset[dataset.train_val_test_idx['val']]
    peptides_test = dataset[dataset.train_val_test_idx['test']]
else:
    # Create train, val, test splits manually
    num_train = int(0.8 * len(dataset))
    num_val = int(0.1 * len(dataset))
    num_test = len(dataset) - num_train - num_val
    peptides_train, peptides_val, peptides_test = torch.utils.data.random_split(
        dataset, [num_train, num_val, num_test]
    )

batch_size = 32
train_loader = pyg.loader.DataLoader(peptides_train, batch_size=batch_size, shuffle=True)
val_loader = pyg.loader.DataLoader(peptides_val, batch_size=batch_size, shuffle=False)
test_loader = pyg.loader.DataLoader(peptides_test, batch_size=batch_size, shuffle=False)

# Check number of classes and label distribution
if hasattr(dataset, 'num_tasks'):
    num_classes = dataset.num_tasks
elif hasattr(dataset, 'num_classes'):
    num_classes = dataset.num_classes
else:
    # Assume binary classification if not specified
    num_classes = 1
print(f"Number of classes (tasks): {num_classes}")

all_labels = np.concatenate([data.y.numpy() for data in dataset], axis=0)
label_distribution = np.mean(all_labels, axis=0)
print(f"Label distribution: {label_distribution}")

# Retrieve the number of node features
# (Some PyG datasets use dataset.num_node_features or
#  you may inspect the first data.x.size(1) for the correct dimension.)
num_node_features = dataset.num_node_features if hasattr(dataset, 'num_node_features') \
                   else peptides_train[0].x.size(-1)
print(f"Number of node features: {num_node_features}")

# ----------------------------
# 3. Instantiate the model, optimizer, and loss function
# ----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphTransformerModel(
    in_channels=num_node_features,
    hidden_channels=64,
    out_channels=num_classes,  # multi-task or single-task
    heads=4,
    dropout=0.5
).to(device)

# If this is a multi-label classification problem, use BCEWithLogitsLoss
# If single-label multi-class, use CrossEntropyLoss
# If regression, use MSELoss, etc.
# The Peptides-func dataset is typically multi-task classification, so BCEWithLogitsLoss is common:
criterion = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ----------------------------
# 4. Define Training and Evaluation Routines
# ----------------------------
def train_one_epoch(loader):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        data.x = data.x.float()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        
        # data.y could have shape [batch_size, num_classes] for multi-label
        # Make sure to cast to float for BCEWithLogitsLoss
        loss = criterion(out, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        data.x = data.x.float()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.float())
        total_loss += loss.item()
    return total_loss / len(loader)

# ----------------------------
# 5. Training Loop
# ----------------------------
best_val_loss = float('inf')
num_epochs = 30

for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(train_loader)
    val_loss = evaluate(val_loader)
    
    # Checkpoint if validation improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
    
    print(f"Epoch [{epoch}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}")

# ----------------------------
# 6. Testing the Best Model
# ----------------------------
model.load_state_dict(torch.load("best_model.pt"))
test_loss = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}")


Number of classes (tasks): 10
Label distribution: [0.08884393 0.03540881 0.06419571 0.06226432 0.6272418  0.19755358
 0.10687023 0.18412581 0.01995769 0.2598179 ]
Number of node features: 9
Epoch [1/30] Train Loss: 0.3816, Val Loss: 0.3673
Epoch [2/30] Train Loss: 0.3627, Val Loss: 0.3721
Epoch [3/30] Train Loss: 0.3576, Val Loss: 0.3864
Epoch [4/30] Train Loss: 0.3550, Val Loss: 0.3862
Epoch [5/30] Train Loss: 0.3526, Val Loss: 0.3811
Epoch [6/30] Train Loss: 0.3507, Val Loss: 0.3777
Epoch [7/30] Train Loss: 0.3482, Val Loss: 0.3639
Epoch [8/30] Train Loss: 0.3471, Val Loss: 0.3665
Epoch [9/30] Train Loss: 0.3448, Val Loss: 0.3673
Epoch [10/30] Train Loss: 0.3432, Val Loss: 0.3786
Epoch [11/30] Train Loss: 0.3416, Val Loss: 0.3574
Epoch [12/30] Train Loss: 0.3397, Val Loss: 0.3507
Epoch [13/30] Train Loss: 0.3382, Val Loss: 0.3511
Epoch [14/30] Train Loss: 0.3375, Val Loss: 0.3511
Epoch [15/30] Train Loss: 0.3361, Val Loss: 0.3535
Epoch [16/30] Train Loss: 0.3351, Val Loss: 0.3552
Epo

/local/u12754_3976534/ipykernel_913727/1693741433.py:183: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


Test Loss: 0.3483
